In [90]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import numpy as np
import pandas as pd
import math

from ipywidgets import interact,interactive,fixed,interact_manual
import ipywidgets as widgets


## Nozzle Plotter

Input quantities:

- throat radius
- throat curve radius
- expansion half-angle
- ratio of exit area to throat area



In [138]:
class Nozzle:
    def __init__(self,r_t,r_1,alpha,epsilon):
        self.r_t = r_t # radius of throat
        self.r_1 = r_1 # radius of throat curvature
        self.alpha = alpha # expansion half-angle
        self.epsilon = epsilon # throat to exit area ratio
        
        self.recalc() # get derived quantities for initial values
        
    def recalc(self):
        # Establish coordinates for arcs
        self.arcCenter = (0,self.r_t + self.r_1)
        self.arclCenter = (0,-(self.r_t + self.r_1))
        
        # Establish coordinates for wall lines
        #n_x and n_y are coordinates for the entry point
        self.n_x = self.r_1*math.sin(math.radians(self.alpha))
        self.n_y = self.r_t+(self.r_1*(1-math.cos(math.radians(self.alpha))))
        
        #e_x and e_y are coordinates for the exit point
        self.r_e = math.sqrt(self.epsilon)*self.r_t
        self.e_y = self.r_e
        self.e_x = self.n_x + (self.r_e-self.n_y)/math.tan(math.radians(self.alpha))
        
    def newParams(self,r_t,r_1,alpha,epsilon):
        self.r_t = r_t
        self.r_1 = r_1
        self.alpha = alpha # expansion half-angle
        self.epsilon = epsilon # throat to exit area ratio
        
        self.recalc() # get derived quantities for initial values
        self.draw() # draw new figure
        
    def draw(self):
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111)
        
        axes = plt.gca()
        axes.set_xlim([-1,9])
        axes.set_ylim([-3,3])
        
        figh = 2*self.r_1
        figw = 2*self.r_1
        arc = mpatches.Arc(xy=self.arcCenter,width=figw,height=figh,theta1=180, theta2=270+self.alpha)
        arcLow = mpatches.Arc(xy=self.arclCenter,width=figw,height=figh,theta1=90-self.alpha, theta2=180)
        
        ax.add_patch(arc)
        ax.add_patch(arcLow)
        
        ax.plot([self.n_x,self.e_x],[self.n_y,self.e_y])
        ax.plot([self.n_x,self.e_x],[-self.n_y,-self.e_y])
        
        plt.show()

In [139]:
schnozz = Nozzle(0.5,1,15,5)

w_tr = widgets.FloatSlider(min=0.1,
                           max=1, 
                           step=0.05,
                           value=0.5,
                           continuous_update=False,
                          description = 'Throat Radius') # throat radius widget
w_tc = widgets.FloatSlider(min=0.1, 
                           max=1, 
                           step =0.05,
                           value=0.7,
                           continuous_update=False,
                          description = 'Curve Radius') # throat curve radius widget
w_a = widgets.FloatSlider(min=0.1,
                          max=30, 
                          step=0.1,
                           continuous_update=False,
                         description='Wall Angle') # wall angle widget
w_e = widgets.FloatSlider(min=1.1, 
                          max=10, 
                          step=0.1,
                           continuous_update=False,
                         description='Area Ratio') # area ratio widget
ui = widgets.VBox([w_tr,w_tc,w_a,w_e])

out = widgets.interactive_output(schnozz.newParams, {'r_t':w_tr,'r_1':w_tc,'alpha':w_a,'epsilon':w_e})

display(ui,out)

Output()

Reference material:
http://seitzman.gatech.edu/classes/ae6450/nozzle_geometries.pdf